# Linear

## Setup (packages and data)

In [1]:
if (!'bigsnpr' %in% installed.packages()) install.packages('bigsnpr')
suppressPackageStartupMessages({
    library(dplyr)
    library(data.table)
    library(bigsnpr)
    library(Rcpp)
    library(RcppArmadillo)
    library(bigparallelr)
    library(foreach)
    library(Matrix)
})

In [ ]:
registerDoParallel(cores=8)

In [ ]:
sourceCpp('Package/linear.cpp')
sourceCpp('Package/utils.cpp')

# SPLENDID

In [ ]:
plink = snp_attach('arrays.rds')

# print('map')
aou_map = fread('aou_ukb_map_final.txt')

related = fread('relatedness_flagged_samples.tsv')
pc_scores = fread('aou_pca.sscore') %>% select(FID=`#FID`, IID, PC=contains('SCORE'))


In [ ]:
for (trait in c('T2D', 'Breast', 'Prostate')) {
    print(trait)
    # data
    full_summaries = readRDS(paste0(trait, '/', trait, '_summaries.RDS'))
    
    phenotype = fread(paste0(trait, '_allpheno.tsv'))
    names(phenotype)[1:3] = c('FID', 'IID', 'Y')
    covariates = fread(paste0(trait, '_cov.tsv'))
    
    data = plink$fam %>%
    select(FID=family.ID, IID=sample.ID) %>%
    left_join(phenotype) %>% 
    left_join(covariates) %>%
    left_join(pc_scores) %>%
    filter(!(IID %in% related$sample_id)) %>%
    na.omit()

    # standardize
    ind_full = which((data$FID %in% plink$fam$fam) & (data$IID %in% plink$fam$samp))

    Y = data$Y[ind_full]
    Y_mean = mean(Y)
    Y_sd = sd(Y)
    Y_std = (Y - Y_mean) / Y_sd

    Z = data[ind_full,] %>%
    dplyr::select(contains(c('age', 'female', paste0('PC', 1:20)))) %>% 
    data.matrix()

    Z_mean = colMeans(Z)
    Z_sd = apply(Z, 2, sd)

    for (k in 1:ncol(Z)) {
        Z[,k] = (Z[,k] - Z_mean[k]) / Z_sd[k]
    }

    Z = cbind(intercept=1, Z)

    E = Z[,paste0('PC', 1:5)]
    
    # run CMSA
    for (K in 1:5) {
        if (!file.exists(paste0(trait, '/cross_fold', K, '.RDS'))) {
            print(K)
            train_fold = read.table(paste0(trait, '/', trait, '_train_fold.txt'), header=F)[,K]
            test_fold = read.table(paste0(trait, '/', trait, '_test_fold.txt'), header=F)[,K]

            full_summary0 = full_summaries[[K]]
            full_summary = data.frame(
                marker.ID=full_summary0$snps,
                XY_norm=full_summary0$XY_norm,
                XEY_norm=full_summary0$XEY_norm,
                X_scale=full_summary0$X_scale,
                XE_scale=full_summary0$XE_scale
            )

            map_match = plink$map %>%
                inner_join(full_summary) %>%
                filter(marker.ID %in% aou_map$aou_code)
            rm(full_summary0); gc()

            aou_ukb_snps = map_match$marker.ID

            # subset indices
            ind_row_train = which((plink$fam$fam %in% data$FID) & (plink$fam$samp %in% data$IID) &
                                  (plink$fam$samp %in% train_fold))
            ind_row_test = which((plink$fam$fam %in% data$FID) & (plink$fam$samp %in% data$IID) &
                                 (plink$fam$samp %in% test_fold))

            ind_col = which(plink$map$marker.ID %in% aou_ukb_snps)

            ind_train = which(data$IID[ind_full] %in% train_fold)
            ind_test = which(data$IID[ind_full] %in% test_fold)

            cat('ind_row_train: ', length(ind_row_train), '\n')
            cat('ind_row_test: ', length(ind_row_test), '\n')
            cat('ind_col: ', length(ind_col), '\n')
            cat('ind_test: ', length(ind_train), '\n')
            cat('ind_train: ', length(ind_test), '\n')

            cat(nrow(plink$map), 'SNPs in bigsnpr \n')
            cat(length(full_summary$marker.ID), 'SNPs in summaries \n')
            cat(length(ind_col), 'SNPs kept \n')
            cat(sum(plink$map$marker.ID[ind_col] %in% plink$map$marker.ID), 'SNPs overlap with bigsnpr \n')
            cat(sum(plink$map$marker.ID[ind_col] %in% full_summary$marker.ID), 'SNPs overlap with summaries \n')

            cat('plink and data training IDs: ', all.equal(plink$fam$samp[ind_row_train], data$IID[ind_full[ind_train]]), '\n')
            cat('plink and data testing IDs: ', all.equal(plink$fam$samp[ind_row_test], data$IID[ind_full[ind_test]]), '\n')
            cat('plink and data SNPs: ', all.equal(plink$map$marker.ID[ind_col], map_match$marker.ID), '\n')

            Y_train = Y_std[ind_train]
            Y_tun = Y_std[ind_test]

            Z_train = Z[ind_train,]
            Z2 = colSums(Z_train^2)
            Z_tun = Z[ind_test,]

            E_train = E[ind_train,]
            E_tun = E[ind_test,]

            ############
            ### L0L1 ###
            ############
            # Parameters
            print('meta-analysis')
            meta_analysis = fread(paste0(trait, '_interaction_gwas.txt'))

            meta_match = map_match %>%
            left_join(meta_analysis, by=c('marker.ID'='rsid'))
            meta_match$P_het[is.na(meta_match$P_het)] = 1
            rm(map_match); gc()

            grouped_list = list()
            grouped_list[['5e-6']] = as.numeric(meta_match$P_het) <= 5e-6
            grouped_list[['5e-8']] = as.numeric(meta_match$P_het) <= 5e-8
            grouped_list[['0']] = rep(F, length(ind_col))
            lapply(grouped_list, sum)

            grid = expand.grid(pval = c('0', '5e-8', '5e-6'), lambda_ix = 1:5) 
            grid$n_grouped = sapply(grid$pval, function(x) sum(grouped_list[[as.character(x)]])) 

            grid = grid %>%
            filter(pval=='0' | n_grouped>0) %>%
            select(-n_grouped)
            # print(grid)

            rm(meta_analysis); gc()

            lambda1_vec = exp(seq(log(1e-2), log(1e-4), length.out=5))

            # combine all results for one fold
            out = foreach(i = 1:nrow(grid)) %dopar% {
                pval = as.character(grid[i,1])
                lambda_ix = grid[i,2]
                lambda1 = lambda1_vec[lambda_ix]

                grouped = grouped_list[[pval]]
                # print(sum(grouped))

                XE = meta_match$XE_scale * grouped + meta_match$X_scale * (1 - grouped)
                XEY = meta_match$XEY_norm * grouped + meta_match$XY_norm * (1 - grouped)
                lambda0_max = max(XE * pmax(abs(XEY/XE) - 0.5*lambda1/XE, 0))^2
                rm(XE, XEY); gc()

                lambda_grid = expand.grid(
                  lambda0_G = exp(seq(log(lambda0_max*0.95), log(lambda0_max*5e-4), length.out=50)),
                  lambda1_G = lambda1, 
                  lambda2_G = 0
                ) %>%
                  arrange(desc(lambda1_G), desc(lambda0_G)) %>%
                  data.matrix()

                test_L0 = cdfit_gaussian_par(
                  BM = plink$genotypes$copy(code = c(0, 1, 2, rep(0, 253))),
                  y = Y_train,
                  rowInd = ind_row_train,
                  colInd = ind_col,
                  Z = Z_train, 
                  E = E_train,
                  y_tun = Y_tun,
                  rowInd_val = ind_row_test,
                  Z_tun = Z_tun,
                  E_tun = E_tun,
                  lambda = lambda_grid,
                  Z2 = Z2,
                  X_scale = meta_match$X_scale,
                  XE_scale = meta_match$XE_scale,
                  XY_norm = meta_match$XY_norm,
                  XEY_norm = meta_match$XEY_norm,
                  grouped = grouped,
                  lambda_ix = as.character(lambda_ix),
                  pval = pval,
                  tol = 1e-3,
                  maxit = 100,
                  dfmax = 1000,
                  n_abort = 5,
                  nlam_min = 10,
                  ncheck = 1
                )

                ix_min = which.min(test_L0$metrics)
                ix_keep = (ix_min-1)*6 + 1:6

                message = paste0(pval, '_', lambda_ix, '_done: ', 
                                 test_L0$nb_active[which.min(test_L0$metrics)], ' ', 
                                 test_L0$nb_interact[which.min(test_L0$metrics)])
                print(message)
                print(Matrix::colSums(test_L0$beta[,ix_keep] != 0), na.rm=T)

                list(
                  beta = test_L0$beta[,ix_keep],
                  ix_best = which.min(test_L0$metrics),
                  nb_active = test_L0$nb_active, 
                  nb_interact = test_L0$nb_interact,
                  metrics = test_L0$metrics,
                  snps = plink$map$marker.ID[ind_col]
                )
            }

            saveRDS(out, paste0(trait, '/cross_fold', K, '.RDS'))
            print(lapply(out, FUN=function(x) Matrix::colSums(x$beta != 0)))

            print(paste0(K, ' done!'))
        }
    }
    system(paste0('gsutil -m cp ', trait, '/cross*RDS ${WORKSPACE_BUCKET}/data/Binary/', trait, '_CV/Background/'), intern=T)
}

## CMSA

In [1]:
for (trait in c('T2D', 'Breast', 'Prostate')) {
    print(trait)
    
    beta = NULL
    n_folds = 0
    for (fold in 1:5) {
        print(fold)
        cross = readRDS(paste0(trait, '/cross_fold', fold, '.RDS'))

        best_par = which.min(unlist(lapply(cross, FUN=function(x) min(x$metrics, na.rm=T))))

        beta_cv = data.matrix(cross[[best_par]]$beta)
        beta_cv[is.na(beta_cv)] = 0
        beta_cv[abs(beta_cv) < 1e-20] = 0
        if (cross[[best_par]]$nb_interact[which.min(cross[[best_par]]$metrics)] == sum(rowSums(abs(beta_cv[,2:6])>0) > 0)) {
            rownames(beta_cv) = cross[[best_par]]$snp

            cat(cross[[best_par]]$nb_active[which.min(cross[[best_par]]$metrics)], sum(beta_cv[,1] != 0), '\n'); 
            cat(cross[[best_par]]$nb_interact[which.min(cross[[best_par]]$metrics)], sum(rowSums(abs(beta_cv[,2:6])>0) > 0), '\n'); 

            if (is.null(beta)) {
                beta = beta_cv
            } else {
                beta = beta + beta_cv
            }
            n_folds = n_folds + 1
        } else {
            cat('SKIP ', fold, '\n')
        }

    }
    beta = beta / n_folds

    ix_keep = which(rowSums(abs(beta) > 0) > 0)

    beta = beta[ix_keep,]
    print(dim(beta))
    print(colSums(beta != 0))
    saveRDS(beta, paste0(trait, '/', trait, '_beta_cv.RDS'))
}

SyntaxError: invalid syntax (1533646430.py, line 1)

# Logistic

## Setup (packages and data)

In [1]:
if (!'bigsnpr' %in% installed.packages()) install.packages('bigsnpr')
suppressPackageStartupMessages({
    library(dplyr)
    library(data.table)
    library(bigsnpr)
    library(Rcpp)
    library(RcppArmadillo)
    library(bigparallelr)
    library(foreach)
    library(Matrix)
})

In [ ]:
registerDoParallel(cores=4)

In [2]:
sourceCpp('Package/logistic.cpp')
sourceCpp('Package/utils.cpp')

## SPLENDID (Logistic)

In [ ]:
plink = snp_attach('arrays.rds')

# print('map')
aou_map = fread('aou_ukb_map_final.txt')

related = fread('relatedness_flagged_samples.tsv')
pc_scores = fread('aou_pca.sscore') %>% select(FID=`#FID`, IID, PC=contains('SCORE'))


In [ ]:
trait='Breast'

full_summaries = readRDS(paste0(trait, '/', trait, '_summaries.RDS'))

summary(full_summaries[[1]])

In [ ]:
full_summaries[[1]] %>% slice_max(XY_norm, n=5)

In [ ]:
for (trait in c('T2D', 'Breast', 'Prostate')) {
    print(trait)
    # data
    full_summaries = readRDS(paste0(trait, '/', trait, '_summaries.RDS'))
    
    phenotype = fread(paste0(trait, '_allpheno.tsv'))
    names(phenotype)[1:3] = c('FID', 'IID', 'Y')
    covariates = fread(paste0(trait, '_cov.tsv'))
    
    data = plink$fam %>%
    select(FID=family.ID, IID=sample.ID) %>%
    left_join(phenotype) %>% 
    left_join(covariates) %>%
    left_join(pc_scores) %>%
    filter(!(IID %in% related$sample_id)) %>%
    na.omit()

    # standardize
    ind_full = which((data$FID %in% plink$fam$fam) & (data$IID %in% plink$fam$samp))

    Y_std = data$Y[ind_full]
    

    Z = data[ind_full,] %>%
    dplyr::select(contains(c('age', 'female', paste0('PC', 1:20)))) %>% 
    data.matrix()

    Z_mean = colMeans(Z)
    Z_sd = apply(Z, 2, sd)

    for (k in 1:ncol(Z)) {
        Z[,k] = (Z[,k] - Z_mean[k]) / Z_sd[k]
    }

    Z = cbind(intercept=1, Z)

    E = Z[,paste0('PC', 1:5)]
    
    # run CMSA
    for (K in 1:5) {
        if (!file.exists(paste0(trait, '/cross_fold', K, '.RDS'))) {
            t0=Sys.time()
            print(K)
            train_fold = read.table(paste0(trait, '/', trait, '_train_fold.txt'), header=F)[,K]
            test_fold = read.table(paste0(trait, '/', trait, '_test_fold.txt'), header=F)[,K]

            full_summary0 = full_summaries[[K]]
            full_summary = data.frame(
                marker.ID=full_summary0$snps,
                XY_norm=full_summary0$XY_norm,
                XEY_norm=full_summary0$XEY_norm,
                X_scale=full_summary0$X_scale,
                XE_scale=full_summary0$XE_scale
            )

            map_match = plink$map %>%
                inner_join(full_summary) %>%
                filter(marker.ID %in% aou_map$aou_code)
            rm(full_summary0); gc()

            aou_ukb_snps = map_match$marker.ID

            # subset indices
            ind_row_train = which((plink$fam$fam %in% data$FID) & (plink$fam$samp %in% data$IID) &
                                  (plink$fam$samp %in% train_fold))
            ind_row_test = which((plink$fam$fam %in% data$FID) & (plink$fam$samp %in% data$IID) &
                                 (plink$fam$samp %in% test_fold))

            ind_col = which(plink$map$marker.ID %in% aou_ukb_snps)

            ind_train = which(data$IID[ind_full] %in% train_fold)
            ind_test = which(data$IID[ind_full] %in% test_fold)

            cat('ind_row_train: ', length(ind_row_train), '\n')
            cat('ind_row_test: ', length(ind_row_test), '\n')
            cat('ind_col: ', length(ind_col), '\n')
            cat('ind_test: ', length(ind_train), '\n')
            cat('ind_train: ', length(ind_test), '\n')

            cat(nrow(plink$map), 'SNPs in bigsnpr \n')
            cat(length(full_summary$marker.ID), 'SNPs in summaries \n')
            cat(length(ind_col), 'SNPs kept \n')
            cat(sum(plink$map$marker.ID[ind_col] %in% plink$map$marker.ID), 'SNPs overlap with bigsnpr \n')
            cat(sum(plink$map$marker.ID[ind_col] %in% full_summary$marker.ID), 'SNPs overlap with summaries \n')

            cat('plink and data training IDs: ', all.equal(plink$fam$samp[ind_row_train], data$IID[ind_full[ind_train]]), '\n')
            cat('plink and data testing IDs: ', all.equal(plink$fam$samp[ind_row_test], data$IID[ind_full[ind_test]]), '\n')
            cat('plink and data SNPs: ', all.equal(plink$map$marker.ID[ind_col], map_match$marker.ID), '\n')

            Y_train = Y_std[ind_train]
            Y_tun = Y_std[ind_test]
            
            Z_train = Z[ind_train,]
            Z2 = colSums(Z_train^2)
            Z_tun = Z[ind_test,]
            
            fit_glm = glm(Y_train ~ 0 + Z_train, family='binomial')
            alpha0 = c(data.matrix(coef(fit_glm)))
            Y_train = Y_train - plogis(Z_train %*% alpha0)


            E_train = E[ind_train,]
            E_tun = E[ind_test,]

            ############
            ### L0L1 ###
            ############
            # Parameters
            print('meta-analysis')
            meta_analysis = fread(paste0(trait, '_interaction_gwas.txt'))

            meta_match = map_match %>%
            left_join(meta_analysis, by=c('marker.ID'='rsid'))
            meta_match$P_het[is.na(meta_match$P_het)] = 1
            rm(map_match); gc()

            grouped_list = list()
            grouped_list[['5e-6']] = as.numeric(meta_match$P_het) <= 5e-6
            grouped_list[['5e-8']] = as.numeric(meta_match$P_het) <= 5e-8
            grouped_list[['0']] = rep(F, length(ind_col))
            lapply(grouped_list, sum)

            grid = expand.grid(pval = c('0', '5e-8', '5e-6'), lambda_ix = 1:5) 
            grid$n_grouped = sapply(grid$pval, function(x) sum(grouped_list[[as.character(x)]])) 

            grid = grid %>%
            filter(pval=='0' | n_grouped>0) %>%
            select(-n_grouped)
            # print(grid)

            rm(meta_analysis); gc()

            lambda1_vec = exp(seq(log(1e-2), log(1e-4), length.out=5))
            out = list()
            for (i in 1:nrow(grid)) {
                if (!file.exists(paste0(trait, '/fit_fold', K, '_', i, '.RDS'))) {
                    pval = as.character(grid[i,1])
                    lambda_ix = grid[i,2]
                    lambda1 = lambda1_vec[lambda_ix]

                    grouped = grouped_list[[pval]]
                    # print(sum(grouped))

                    XE = meta_match$XE_scale * grouped + meta_match$X_scale * (1 - grouped)
                    XEY = meta_match$XEY_norm * grouped + meta_match$XY_norm * (1 - grouped)
                    lambda0_max = max(XE * pmax(abs(XEY/XE) - 0.5*lambda1/XE, 0))^2
                    rm(XE, XEY); gc()

                    if (lambda0_max > 0) {
                        lambda_grid = expand.grid(
                          lambda0_G = exp(seq(log(lambda0_max*0.95), log(lambda0_max*5e-4), length.out=50)),
                          lambda1_G = lambda1, 
                          lambda2_G = 0
                        ) %>%
                          arrange(desc(lambda1_G), desc(lambda0_G)) %>%
                          data.matrix()
                    } else {
                        lambda_grid = expand.grid(
                          lambda0_G = 0,
                          lambda1_G = lambda1, 
                          lambda2_G = 0
                        ) %>%
                          arrange(desc(lambda1_G), desc(lambda0_G)) %>%
                          data.matrix()
                    }


                    system(paste0('echo START > ', trait, '_', K, '_', i, '.txt'))
                    test_L0 = cdfit_binomial_par(
                        BM = plink$genotypes$copy(code = c(0, 1, 2, rep(0, 253))),
                        y = Y_train,
                        rowInd = ind_row_train,
                        colInd = ind_col,
                        Z = Z_train, 
                        E = E_train,
                        y_tun = Y_tun,
                        rowInd_val = ind_row_test,
                        Z_tun = Z_tun,
                        E_tun = E_tun,
                        alpha0 = alpha0,
                        weights = rep(1, length(ind_col)),
                        lambda = lambda_grid,
                        Z2 = Z2,
                        X_scale = meta_match$X_scale,
                        XE_scale = meta_match$XE_scale,
                        XY_norm = meta_match$XY_norm,
                        XEY_norm = meta_match$XEY_norm,
                        grouped = grouped,
                        lambda_ix = as.character(lambda_ix),
                        pval = pval,
                        tol = 1e-3,
                        maxit=100,
                        dfmax = 20,
                        n_abort = 2,
                        nlam_min = 5,
                        ncheck = 1,
                        verbose=T
                    )

                    ix_min = which.min(test_L0$metrics)
                    ix_keep = (ix_min-1)*6 + 1:6

                    message = paste0(pval, '_', lambda_ix, '_done: ', 
                                     test_L0$nb_active[which.min(test_L0$metrics)], ' ', 
                                     test_L0$nb_interact[which.min(test_L0$metrics)])
                    print(message)
                    print(Matrix::colSums(test_L0$beta[,ix_keep] != 0), na.rm=T)
                    system(paste0('echo DONE >> ', trait, '_', K, '_', i, '.txt'))

                    saveRDS(test_L0, paste0(trait, '/fit_fold', K, '_', i, '.RDS'))

                    out[[i]] = list(
                      beta = test_L0$beta[,ix_keep],
                      ix_best = which.min(test_L0$metrics),
                      nb_active = test_L0$nb_active, 
                      nb_interact = test_L0$nb_interact,
                      metrics = test_L0$metrics,
                      snps = plink$map$marker.ID[ind_col]
                    )
                }
            }
            t1=Sys.time()
                                    
            saveRDS(out, paste0(trait, '/cross_fold', K, '.RDS'))
            print(lapply(out, FUN=function(x) Matrix::colSums(x$beta != 0)))

            print(paste0(K, ' done!'))
            print(t1-t0)
        }
    }
}

## CMSA

In [ ]:
for (trait in c('T2D', 'Breast', 'Prostate')) {
    print(trait)
    
    beta = NULL
    n_folds = 0
    for (fold in 1:5) {
        print(fold)
        cross = readRDS(paste0(trait, '/cross_fold', fold, '.RDS'))

        best_par = which.min(unlist(lapply(cross, FUN=function(x) min(x$metrics, na.rm=T))))

        beta_cv = data.matrix(cross[[best_par]]$beta)
        beta_cv[is.na(beta_cv)] = 0
        beta_cv[abs(beta_cv) < 1e-20] = 0
        if (cross[[best_par]]$nb_interact[which.min(cross[[best_par]]$metrics)] == sum(rowSums(abs(beta_cv[,2:6])>0) > 0)) {
            rownames(beta_cv) = cross[[best_par]]$snp

            cat(cross[[best_par]]$nb_active[which.min(cross[[best_par]]$metrics)], sum(beta_cv[,1] != 0), '\n'); 
            cat(cross[[best_par]]$nb_interact[which.min(cross[[best_par]]$metrics)], sum(rowSums(abs(beta_cv[,2:6])>0) > 0), '\n'); 

            if (is.null(beta)) {
                beta = beta_cv
            } else {
                beta = beta + beta_cv
            }
            n_folds = n_folds + 1
        } else {
            cat('SKIP ', fold, '\n')
        }

    }
    beta = beta / n_folds

    ix_keep = which(rowSums(abs(beta) > 0) > 0)

    beta = beta[ix_keep,]
    print(dim(beta))
    print(colSums(beta != 0))
    saveRDS(beta, paste0(trait, '/', trait, '_beta_cv.RDS'))
}